In [1]:
import QuantLib as ql
import numpy as np
import pandas as pd
from datetime import datetime
import model_settings
from model_settings import ms
from itertools import product


pricing settings:
Actual/365 (Fixed) day counter
New York stock exchange calendar
compounding: continuous
frequency: annual



In [2]:
calculation_datetime = datetime.today()
calculation_date = ql.Date(
    calculation_datetime.day,
    calculation_datetime.month,
    calculation_datetime.year
)

r = 0.05
g = 0.02
s = 100.00
k = 95.00
w = 'put'
t = 360

kappa = 0.2
theta = 0.02
rho = -0.75
eta = 0.5
v0 = 0.01 

past_fixings = 0
averaging_frequency = 5

In [3]:
my_geometric = ms.ql_asian_price(
            s,k,t,r,g,calculation_datetime, w,
            'geometric',past_fixings,averaging_frequency,
            kappa,theta,rho,eta,v0
            )

my_arithmetic = ms.ql_asian_price(
            s,k,t,r,g,calculation_datetime, w,
            'arithmetic',past_fixings,averaging_frequency,
            kappa,theta,rho,eta,v0
            )

In [4]:
vanilla = ms.ql_heston_price(
    s,k,t,r,g,w, 
    kappa, theta, rho, eta, v0, 
    calculation_datetime
)
print(f"\nvanilla: {vanilla}\n\ngeometric: {my_geometric}\n\narithmetic: {my_arithmetic}\n")


vanilla: 1.257354183177545

geometric: 0.6856184721614448

arithmetic: 0.6380115307796965



In [5]:
# K = np.arange(int(s*0.5),int(s)*1.5+1,5).astype(int).tolist()
K = np.arange(s*0.8,s*1.2,5).astype(int).tolist()
averaging_frequencies = [15,180]
T = [180,360]
past_fixings = [0]

features = pd.DataFrame(
    product(
        [s],K,T,
        averaging_frequencies,
        ['call','put'],
        ['geometric','arithmetic'],
        past_fixings,
        [theta],
        [kappa],
        [rho],
        [eta],
        [v0],
        [r],
        [g],
        [calculation_datetime]
    ),
    columns = [ 
        'spot_price',
        'strike_price',
        'days_to_maturity',
        'averaging_frequency',
        'w',
        'averaging_type',
        'past_fixings',
        'theta','kappa','rho','eta','v0',
        'risk_free_rate',
        'dividend_rate',
        'calculation_date',
    ]
)

features['vanilla_price'] = ms.vector_heston_price(features)
features['asian_price'] = ms.vector_asian_price(features)

In [6]:
pd.set_option("display.float_format", '{:.6f}'.format)
pd.set_option("display.max_rows",None)
features = features[
    (
        (features['averaging_type']=='arithmetic')
        &
        (features['w']=='put')
    )
]
features[
    [
        'vanilla_price', 'asian_price','spot_price', 'strike_price', 'days_to_maturity', 'averaging_frequency',
        'w', 'averaging_type', 'past_fixings', 'kappa', 'theta', 'rho', 'eta',
        'v0', 'risk_free_rate', 'dividend_rate', 'calculation_date',

    ]
].sort_values(by=['strike_price','days_to_maturity','averaging_frequency','averaging_type','w']).reset_index(drop=True)

,vanilla_price,asian_price,spot_price,strike_price,days_to_maturity,averaging_frequency,w,averaging_type,past_fixings,kappa,theta,rho,eta,v0,risk_free_rate,dividend_rate,calculation_date
0,0.188506,0.013372,100.000000,80,180,15,put,arithmetic,0,0.200000,0.020000,-0.750000,0.500000,0.010000,0.050000,0.020000,2024-10-16 09:25:25.425185
1,0.188506,0.176066,100.000000,80,180,180,put,arithmetic,0,0.200000,0.020000,-0.750000,0.500000,0.010000,0.050000,0.020000,2024-10-16 09:25:25.425185
2,0.478264,0.088131,100.000000,80,360,15,put,arithmetic,0,0.200000,0.020000,-0.750000,0.500000,0.010000,0.050000,0.020000,2024-10-16 09:25:25.425185
3,0.478264,0.258529,100.000000,80,360,180,put,arithmetic,0,0.200000,0.020000,-0.750000,0.500000,0.010000,0.050000,0.020000,2024-10-16 09:25:25.425185
4,0.327197,0.045431,100.000000,85,180,15,put,arithmetic,0,0.200000,0.020000,-0.750000,0.500000,0.010000,0.050000,0.020000,2024-10-16 09:25:25.425185
5,0.327197,0.324528,100.000000,85,180,180,put,arithmetic,0,0.200000,0.020000,-0.750000,0.500000,0.010000,0.050000,0.020000,2024-10-16 09:25:25.425185
6,0.662318,0.180163,100.000000,85,360,15,put,arithmetic,0,0.200000,0.020000,-0.750000,0.500000,0.010000,0.050000,0.020000,2024-10-16 09:25:25.425185
7,0.662318,0.395879,100.000000,85,360,180,put,arithmetic,0,0.200000,0.020000,-0.750000,0.500000,0.010000,0.050000,0.020000,2024-10-16 09:25:25.425185
8,0.552411,0.144071,100.000000,90,180,15,put,arithmetic,0,0.200000,0.020000,-0.750000,0.500000,0.010000,0.050000,0.020000,2024-10-16 09:25:25.425185
9,0.552411,0.536082,100.000000,90,180,180,put,arithmetic,0,0.200000,0.020000,-0.750000,0.500000,0.010000,0.050000,0.020000,2024-10-16 09:25:25.425185


In [7]:
pd.reset_option("display.max_rows")